# Pipeline for a data warehouse
### Data Engineering Capstone Project

#### Project Summary
This is the capstone project of the udacity nanodegree for data engineering. 
The aim of the project is to apply learned skills during the course. This project will show how to load and transform data from four different data sources, load the data in spark apply quality checks and store the data into a star schema so that it can be used for BI apps and ad hoc analysis.

The project follows the follow steps:
* _Step 1_: Scope the Project and Gather Data
* _Step 2_: Explore and Assess the Data
* _Step 3_: Define the Data Model
* _Step 4_: Run ETL to Model the Data
* _Step 5_: Complete Project Write Up

### Step 1: Scope the Project and Gather Data

#### Scope 
As one of the acceptance criteria for this project is to handle at least 1 million rows and two different data sources and file formats, we will use the data sources are provided by Udacity. 
The scope of this project is to create a star schema source-of-truth database so that it can be used for BI apps and ad hoc analysis.

#### Describe and Gather Data 

The main dataset includes data about 
- _Immigration in the United States of America_ from [here](https://www.trade.gov/national-travel-and-tourism-office).

Supplementary datasets provided are:

- U.S. city _demographics_ from [here](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/)

- _Temperature_ data from [here](https://www.trade.gov/national-travel-and-tourism-office)

- Data on _airport codes_ from [here](https://datahub.io/core/airport-codes#data)

### Step 2 - Explore and Asses the Data
The exploration of the used datasets including descriptions of the above mentioned data can be found in  
* `exploration.ipynb`. *

#### Important findings of the exploration

##### Immigrant data:
 -- dates are provided in SAS timestamps and need to be transformed

 -- there are many columns that have a lot of null values or are marked with CIC does not use", hence can be skipped. As the immigrant data is provided in parquet files, we can only read in the columns we need for this project.

 -- as the data is provided in parquet files, we can only load the data we need

 -- We might need helper tables to provide data for the

 --- visa codes: 1 = Business 2 = Pleasure 3 = Student 
 
 --- travel code:  1 = 'Air' 2 = 'Sea' 3 = 'Land' 9 = 'Not 

##### City demographics data:
- the "count" column can be skipped, as it was used for some other analysis
- the "number of veterans" can be also skipped as the main focus lies on the immigrant data.
 
##### Weather data:

- The temperature data is provided on a monthly basis
- The "dt" gives us the the date as a string. We need to transform it to datetime. As it holds data from 1860 on, we only want to keep the rows from 1960 on. 
- We only want to use the rows where the country is US.
- The name of the city is used to map to the weather and then store to weather table in order to get a star schema.

##### Possible next steps:
Connect airport data based on lat and long. Maybe with some margin.
We could also create a helper table to map the ports to cities.

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
The data shall be used for ad hoc queries and BI Apps. Therefore the data shall be represented in a star schema.
The advantage is, that it is easy to query an easy to understand.

![star schema]("./capstone_schema.png")

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

In order to pipe the data into the chosen data model, we will use Spark. This has the advantage that we can do the transformations in memory, before writing the data to the tables.
1. Load only the needed columns from the immigrant files into a spark dataframe. 
2. Transform arrival and departure date to timestamps
3. Create the fact_immigration table from the loaded immigrant files and write to parquet files.
4. Create the dim_immigrant_person table from the loaded immigrant files and write to parquet files.
5. Create the dim_time table from the transformed timestamps
6. Load weather data, filter the weather data by country, as we only need data from the United States from 1960 on and save them as dim_weather
7. Load the city data and write to parquet files as dim_city

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.
The code can be found in `etl.py`

In [ ]:
import etl_capstone

etl_capstone.main()

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

- We are using  _schemas_, when reading the data, with the schema we can assure that the data type is correct. See `schemas.py`
- We are using _dropDuplicates()_ to get rid of duplicate data.
- We are count the number of rows and columns for the output tables. See `utils.py`


#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### FACT IMMIGRATION

| column name| description | type | data source | 
| --- | --- | --- | --- |
|cicid | unique number for an immigrant | integer, not nullable | sas_data  immigration |
|arrival_ts | timestamp  of the arrival date | timestamp | sas_data  immigration: transformed from field "arrdata": SAS numeric |
|departure_ts | timestamp  of the arrival date | timestamp | sas_data  immigration: transformed from field "depdate": SAS numeric |
|i94cit | 3 digit code of origin city | short, not nullable | sas_data  immigration |
|i94res | 3 digit from the country one has travelled | short | sas_data  immigration |
|i94port | 3 char code of origin city in USA| string | sas_data  immigration |
|fltno | flight number of airline that arrived in us | string | sas_data  immigration |

#### DIM TIME

Datasource: all timestamps are taken frome the arrival and departure date of the fact_immigration table

| column name| description | type |
| --- | --- | --- | 
| ts | unix timestamp, not nullable | ts |
| date | date | string |
| year | year | integer |
| month | month | integer |
| weekday | weekday | integer |
| day | day | integer |
| hour | hour | integer |

#### DIM IMMIGRANT
Data Source:  sas_data  immigration

| column name| description | type |
| --- | --- | --- | 
|cicid|  unique number for the immigrants| int |
|biryear| year of birth| int |
|gender| gender of immigrant| string |
|i94visa| Visa codes collapsed into three categories, 1 = Business 2 = Pleasure 3 = Student |int |


#### DIM CITY

Datasource: https://public.opendatasoft.com/explore/dataset/us-cities-demographics

| column name| description | type | 
| --- | --- | --- |
|city_name| name of the city| string |
|state | state | string |
|male_population | number of male population| int |
|male_population | number of female population| int |
|total_population | number of total population| int |
|foreign_born | number of foreign born | int |
|average_household | average number of people in a household | double |
| state_code | code of the state | int |


#### DIM TEMPERATURE

| column name| description | type |
| --- | --- | --- | 
|dt|date|date|
|AverageTemperature| <- |float|
|AverageTemperatureUncertainty|<- |string|
|City|<- | string|
|Country|<- | string|
|year| <- | integer|
|month| <- |integer|
|Langitude| <- |integer|
|Latidue| <- |integer|

#### Step 5: Complete Project Write Up
1. _Clearly state the rationale for the choice of tools and technologies for the project._
    We choose Apache Spark, because
    - we are able to read in only the columns we need from parquet files
    - we are able to do data cleaning and transformation either with dataframes or sql syntax.
    - it provides an easy to use API
    - it can handle a lot of different file formats 
    - it can handle big amounts of data  <br/>
  

2. _Propose how often the data should be updated and why._
    - It depends on the amount of data and how often the data is updated. If the data is updated every week, it makes sense to run the pipeline every week. <br/>


3. _Write a description of how you would approach the problem differently under the following scenarios:_  <br/>

 3a. _The data was increased by 100x._<br/>
    - The bigger the data gets, the more computing power is helpful to process the data. Adding more nodes to our cluster could be a way of dealing with a bigger amount of data.


 3b. _The data populates a dashboard that must be updated on a daily basis by 7am every day._<br/>
    - For scheduled pipelines a tool like [Airflow](https://airflow.apache.org/docs/) can be used. It has the advantage, that it provides a web view, so that also non programmers can check wether a pipeline ran successfully or not.


 3c. _The database needed to be accessed by 100+ people._<br/>
    - There are different cloud solutions availabe for this scenario. We could use [Databricks](https://databricks.com/) it can handle a lot of simultaneasly connections.
